In [1]:
import os
import re

from rdkit import Chem
from openbabel import pybel
import parameter_extractor as pe
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import DrawingOptions

IPythonConsole.molSize = (300, 300)  # Change image size
DrawingOptions.bondLineWidth = 1.8

# Gaussian Input Files Generation
This notebook generates `.gjf` and script files to submit the calculations to gaussian.

This cell creates input files for `SPE_NoRh` structures.

In [2]:
directory = "./out/pending/"

In [3]:
for file in os.listdir(directory):
    path = directory + file
    ligand_number = re.search(r"\d+[a-z]*", file)[0]
    submit_filename = f"l_{ligand_number}_SPE_NoRh"
    submit_file = f"""#!/bin/csh
#$ -N {submit_filename}
#$ -pe smp 14
#$ -q iqtc08.q
#$ -S /bin/csh
#$ -cwd
#$ -o $HOME/javier/{submit_filename}.log
#$ -e $HOME/javier/{submit_filename}.err
#$ -m e  
#$ -M javier.eusamio@ub.edu 

# Load the modules needed
source /etc/profile.d/modules.csh
module load gaussian/g16b01

cd $TMPDIR
cp -pr $HOME/javier/{submit_filename}.gjf .

# Run the job
g16 < {submit_filename}.gjf > {submit_filename}.out

# Copy the results to our home directory
cp {submit_filename}.out $HOME/javier/"""
    
    # Removes COD + Rh from molecule, converts to .gjf and adds job keywords
    comp = pe.ParameterExtractor(path, include_spe_prop=False)
    editable_mol = Chem.RWMol(comp.mol)
    cod_idx = [a.GetIdx() for a in comp.cod]
    remove_atoms = [a for a in editable_mol.GetAtoms() if a.GetIdx() in cod_idx]
    remove_atoms.append(comp.rh)
    remove_atoms = sorted(remove_atoms, key=lambda x: x.GetIdx(), reverse=True)
    for atom in remove_atoms:
        atom.SetAtomMapNum(atom.GetIdx())
        editable_mol.RemoveAtom(atom.GetIdx())
    pdb_block = Chem.MolToPDBBlock(editable_mol)
    pdb = pybel.readstring("pdb", pdb_block)
    gjf = pdb.write("gjf")
    gjf = re.sub(
        r"!Put Keywords Here, check Charge and Multiplicity.\n#\n\n",
        fr"%nprocshared=14\n%mem=8GB\n# nmr=giao b3lyp/def2TZVP pop=nbo scfcyc=200\n\nLigand {ligand_number} SPE NoRh",
        gjf
    )
    with open(f"./gjf/NoRh/l_{ligand_number}_SPE_NoRh.gjf", "w") as f:
        f.write(gjf)
    with open(f"./gjf/NoRh/{submit_filename}", "w", newline='\n') as f:
        f.write(submit_file)
    print(f"Ligand {ligand_number} done!")

Ligand 87 done!
